In [42]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import libraries

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read Datasets

In [44]:
#read input files
companies = pd.read_csv("C:/Users/Rabi/Documents/Data_Science/upgrad/investment_case_study/companies.txt",
                          sep = "\t", encoding = 'unicode_escape' )
rounds2 = pd.read_csv("C:/Users/Rabi/Documents/Data_Science/upgrad/investment_case_study/rounds2.csv", encoding = 'unicode_escape')


# Inspect and Pre-process dataset rounds2

In [45]:
#inspect companies file
print(rounds2.shape)

(114949, 6)


In [46]:
print(rounds2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB
None


In [47]:
print(rounds2.describe())

       raised_amount_usd
count       9.495900e+04
mean        1.042687e+07
std         1.148212e+08
min         0.000000e+00
25%         3.225000e+05
50%         1.680511e+06
75%         7.000000e+06
max         2.127194e+10


In [48]:
print(rounds2.head(5))

                     company_permalink  \
0                  /organization/-fame   
1               /ORGANIZATION/-QOUNTER   
2               /organization/-qounter   
3  /ORGANIZATION/-THE-ONE-OF-THEM-INC-   
4                /organization/0-6-com   

                           funding_round_permalink funding_round_type  \
0  /funding-round/9a01d05418af9f794eebff7ace91f638            venture   
1  /funding-round/22dacff496eb7acb2b901dec1dfe5633            venture   
2  /funding-round/b44fbb94153f6cdef13083530bb48030               seed   
3  /funding-round/650b8f704416801069bb178a1418776b            venture   
4  /funding-round/5727accaeaa57461bd22a9bdd945382d            venture   

  funding_round_code   funded_at  raised_amount_usd  
0                  B  05-01-2015         10000000.0  
1                  A  14-10-2014                NaN  
2                NaN  01-03-2014           700000.0  
3                  B  30-01-2014          3406878.0  
4                  A  19-03-2008      

In [49]:
#standardise the raise_amount_usd to millions by dividing it by 10^6
rounds2.raised_amount_usd = rounds2.raised_amount_usd/(10 ** 6)
rounds2.raised_amount_usd.head(10)

0    10.000000
1          NaN
2     0.700000
3     3.406878
4     2.000000
5          NaN
6     0.041250
7     0.043360
8     0.719491
9     3.000000
Name: raised_amount_usd, dtype: float64

In [50]:
print(round(rounds2.raised_amount_usd.isnull().sum()/len(rounds2), 2))
#rounds2.raised_amount_usd.loc[~rounds2.raised_amount_usd.isnull()]
print(rounds2.raised_amount_usd.describe())
print(rounds2.raised_amount_usd.median())
print(rounds2.raised_amount_usd.mean())
#the data is not normal and higly skewed towards left hence we should not use mean value for imputation


0.17
count    94959.000000
mean        10.426869
std        114.821248
min          0.000000
25%          0.322500
50%          1.680511
75%          7.000000
max      21271.935000
Name: raised_amount_usd, dtype: float64
1.680511
10.426869334502733


In [51]:
#missing values of funding round code can be ignored as it does not add any value to our analysis
#find candidates for converting to categorical variables
print(rounds2.apply(lambda x: len(x.unique())))
#converting characters variable which are categorical
rounds2.funding_round_type = pd.Categorical(rounds2.funding_round_type)
print(rounds2.funding_round_type.head(5))
rounds2.funding_round_code = pd.Categorical(rounds2.funding_round_code)
print(rounds2.funding_round_code.head(5))

#converting funded_at to date variable which is presnelty in dd-mm-yyyy character format
print(rounds2.funded_at.head(5))
rounds2.funded_at = pd.to_datetime(rounds2.funded_at, format = '%d-%m-%Y')
print(rounds2.funded_at.head(5))





company_permalink           90247
funding_round_permalink    114949
funding_round_type             14
funding_round_code              9
funded_at                    5033
raised_amount_usd           22096
dtype: int64
0    venture
1    venture
2       seed
3    venture
4    venture
Name: funding_round_type, dtype: category
Categories (14, object): [angel, convertible_note, debt_financing, equity_crowdfunding, ..., secondary_market, seed, undisclosed, venture]
0      B
1      A
2    NaN
3      B
4      A
Name: funding_round_code, dtype: category
Categories (8, object): [A, B, C, D, E, F, G, H]
0    05-01-2015
1    14-10-2014
2    01-03-2014
3    30-01-2014
4    19-03-2008
Name: funded_at, dtype: object
0   2015-01-05
1   2014-10-14
2   2014-03-01
3   2014-01-30
4   2008-03-19
Name: funded_at, dtype: datetime64[ns]


In [52]:
# the company peralink has data in both upper and lower case. 
#this may cause each of the multiple entries of same data value to be treated distinctly.
#lets say GOOD, Good and good all will be tretaed distictly.
#We want all thee to be treated the asme as good
#converting this all to lower case to bring uniformity. we also observe some accensted character strings
# we fix this. The code used to fix this is attributed to
#https://stackoverflow.com/questions/45871731/removing-special-characters-in-a-pandas-dataframe
print(len(rounds2.company_permalink.str.lower().unique()))
rounds2.company_permalink = rounds2.company_permalink.str.lower()
rounds2.company_permalink = rounds2.company_permalink.str.encode('utf-8').str.decode('ascii', 'ignore')
print(len(rounds2.company_permalink.unique()))

66370
66368


# Inspect and Pre-Process datase companies

In [53]:
print(companies.shape)

(66368, 10)


In [54]:
print(companies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB
None


In [55]:
print(companies.describe())

                            permalink   name                homepage_url  \
count                           66368  66367                       61310   
unique                          66368  66102                       61191   
top     /Organization/Arta-Bioscience  Peach  http://www.askforoffer.com   
freq                                1      4                           5   

       category_list     status country_code state_code       region  \
count          63220      66368        59410      57821        58338   
unique         27296          4          137        311         1092   
top         Software  operating          USA         CA  SF Bay Area   
freq            3995      53034        37601      12900         8804   

                 city  founded_at  
count           58340       51147  
unique           5111        3978  
top     San Francisco  01-01-2012  
freq             3526        2730  


In [56]:
print(len(companies))
print(companies.apply(lambda x: len(x.unique())))
print(companies.founded_at.head(10))

66368
permalink        66368
name             66103
homepage_url     61192
category_list    27297
status               4
country_code       138
state_code         312
region            1093
city              5112
founded_at        3979
dtype: int64
0           NaN
1    04-09-2014
2           NaN
3    01-01-2007
4    01-01-2010
5           NaN
6    01-01-1997
7    01-01-2011
8    01-08-2011
9    01-01-2012
Name: founded_at, dtype: object


In [57]:
#status and country code can be converted to category, founded_at to be converted to date
#permalink is exactly of same length as number of rows.
#This might be candidate for uniqness. will chek firther
companies.status = pd.Categorical(companies.status)
#companies.country_code = pd.Categorical(companies.country_code)

In [58]:
#companies funded at can be converted to date
companies.founded_at = pd.to_datetime(companies.founded_at, format = '%d-%m-%Y', errors='coerce')

In [59]:
#every entry in companies for permalink variable is unique as seen above. Lets see if same is the case
#after converting each to lower
len(companies.permalink.str.lower().unique()) == len(companies)

#lower case the permalink and resolve accented characters
companies.permalink = companies.permalink.str.lower()
companies.permalink = companies.permalink.str.encode('utf-8').str.decode('ascii', 'ignore')




In [60]:
#lets check for missing values
companies.info()
print(round(companies.isnull().sum(), 2))
print(round(companies.isnull().sum()/len(companies), 2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null category
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51038 non-null datetime64[ns]
dtypes: category(1), datetime64[ns](1), object(8)
memory usage: 4.6+ MB
permalink            0
name                 1
homepage_url      5058
category_list     3148
status               0
country_code      6958
state_code        8547
region            8030
city              8028
founded_at       15330
dtype: int64
permalink        0.00
name             0.00
homepage_url     0.08
category_list    0.05
status           0.00
country_code     0.10
state_code       0.13
region           0.12


In [61]:
# # #country_code and category_list is the most important feature for the analysis.
# # #lets find if we can fill these by inspecting values of other varibles

# # #chekcing if city gives a hint of the country_code
(companies[companies.country_code.isnull()]["city"].isnull() == False).sum()

# # #we observe that all the city column for country code NULL is also NULL.
# # #lets check if webpage givees any indication. its common to have country
# # #specific domain names like .au or .in

#checking if we have relevant values to work with
print((companies[companies.country_code.isnull()]["homepage_url"].isnull() == False).sum())

#extract the last toplevel domain
companies["top_lvl_domain"] = companies["homepage_url"].str.extract(r'(\w+)$')

#read in csv files containing mapping of top level domain and country code
#this data was sourced from https://www.ionos.com/digitalguide/domains/domain-extensions/cctlds-a-list-of-every-country-domain/
#https://www.worldstandards.eu/other/tlds/
top_lvl_dom = pd.read_csv("C:/Users/Rabi/Documents/Data_Science/upgrad/investment_case_study/country_url_toplvldomain.csv", encoding = 'unicode_escape')

#merging thedomain lvel info with companies
companies = pd.merge(companies, top_lvl_dom, how='left', left_on='top_lvl_domain', right_on = 'top_lvl_domain')

5552


In [62]:
#checking how many missing country_code can be coded
len(companies[companies.country_code.isnull() * ~companies.code.isnull()].code.index.values)

#getting their index and assinging codes.
idx = companies[companies.country_code.isnull() * ~companies.code.isnull()].code.index.values.tolist()
companies.loc[idx, "country_code"] = companies.loc[idx, "code"].tolist()
companies.country_code = pd.Categorical(companies.country_code)

In [63]:
companies.drop(["top_lvl_domain", "country_name","code"], axis = 1, inplace=True)

In [64]:
companies.info()
print(round(companies.isnull().sum(), 2))
print(round(companies.isnull().sum()/len(companies), 2))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null category
country_code     60011 non-null category
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51038 non-null datetime64[ns]
dtypes: category(2), datetime64[ns](1), object(7)
memory usage: 7.3+ MB
permalink            0
name                 1
homepage_url      5058
category_list     3148
status               0
country_code      6357
state_code        8547
region            8030
city              8028
founded_at       15330
dtype: int64
permalink        0.00
name             0.00
homepage_url     0.08
category_list    0.05
status           0.00
country_code     0.10
state_code       0.13
region           0.1

# We now proceed with analysis asked as per assignment

## Data Cleaning 1

In [65]:
#How many unique companies are present in rounds2?
len(rounds2.company_permalink.unique())


66368

In [66]:
#How many unique companies are present in the companies file?
len(companies.permalink.unique())

66368

In [67]:
#In the companies data frame, which column can be used as the  unique key for each company?
#Write the name of the column.
print(len(companies))
print(companies.apply(lambda x: len(x.unique())))


66368
permalink        66368
name             66103
homepage_url     61192
category_list    27297
status               4
country_code       156
state_code         312
region            1093
city              5112
founded_at        3904
dtype: int64


In [68]:
#Are there any companies in the rounds2 file which are not  present in companies ?
#Answer Y(True)/N(False).

print(len(set(rounds2.company_permalink).difference(set(companies.permalink))) > 0)

False


In [69]:
#Merge the two data frames so that all variables (columns) in the companies frame are
#added to the rounds2 data frame. Name the merged frame master_frame.
#How many observations are present in master_frame?

master_frame = pd.merge(rounds2, companies, how='left', left_on='company_permalink', right_on = 'permalink')
master_frame.info()
round(master_frame.isnull().sum()/len(master_frame), 2)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 114949 entries, 0 to 114948
Data columns (total 16 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null category
funding_round_code         31140 non-null category
funded_at                  114949 non-null datetime64[ns]
raised_amount_usd          94959 non-null float64
permalink                  114949 non-null object
name                       114948 non-null object
homepage_url               108815 non-null object
category_list              111539 non-null object
status                     114949 non-null category
country_code               107015 non-null category
state_code                 104003 non-null object
region                     104782 non-null object
city                       104785 non-null object
founded_at                 94286 non-null datetime64[ns]
dtypes: category(4), datetime64[ns](2), float64(1), object(9)
memor

company_permalink          0.00
funding_round_permalink    0.00
funding_round_type         0.00
funding_round_code         0.73
funded_at                  0.00
raised_amount_usd          0.17
permalink                  0.00
name                       0.00
homepage_url               0.05
category_list              0.03
status                     0.00
country_code               0.07
state_code                 0.10
region                     0.09
city                       0.09
founded_at                 0.18
dtype: float64

## Funding Type Analysis

In [70]:
#calculate avergae investemnts in diffrent FT
round(master_frame.groupby("funding_round_type").raised_amount_usd.mean().sort_values(ascending=False), 2)

funding_round_type
post_ipo_debt            168.70
post_ipo_equity           82.18
secondary_market          79.65
private_equity            73.31
undisclosed               19.24
debt_financing            17.04
venture                   11.75
grant                      4.30
convertible_note           1.45
product_crowdfunding       1.36
angel                      0.96
seed                       0.72
equity_crowdfunding        0.54
non_equity_assistance      0.41
Name: raised_amount_usd, dtype: float64

From the table above it looks like given the constrains of 5 to 15 mn ticket size Spark Fund should invest in ventures.

## Country Analysis

In [71]:
#Spark Funds wants to see the top nine countries which have received the highest total 
#funding (across ALL sectors for the chosen investment type)

group_country = master_frame[master_frame.funding_round_type == "venture"].groupby("country_code")
top9 = round(group_country.raised_amount_usd.sum().sort_values(ascending = False), 2).head(9).to_frame()
top9

,raised_amount_usd
country_code,
USA,422516.84
CHN,40164.30
GBR,20309.70
IND,14394.56
CAN,9585.53
FRA,7298.70
ISR,6937.01
DEU,6346.96
JPN,3398.14


Fro table above we see that th top3 english speaking countries are USA, Great Britain and India.

## Sector Analysis 1

In [72]:
#reading mapping file
main_sector_mapping = pd.read_csv("C:/Users/Rabi/Documents/Data_Science/upgrad/investment_case_study/mapping.csv", encoding = 'unicode_escape')
main_sector_mapping.head(5)

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [73]:
#converting to tabular
main_sector_mapping = pd.melt(main_sector_mapping, id_vars="category_list", var_name = "main_sector")
main_sector_mapping.head(5)

,category_list,main_sector,value
0,NaN,Automotive & Sports,0
1,3D,Automotive & Sports,0
2,3D Printing,Automotive & Sports,0
3,3D Technology,Automotive & Sports,0
4,Accounting,Automotive & Sports,0


In [74]:
#selecting relevant rows for mapping.
main_sector_mapping = main_sector_mapping[main_sector_mapping.value == 1]
main_sector_mapping.drop(["value"] , axis = 1, inplace = True)
main_sector_mapping.sample(5)

,category_list,main_sector
6117,Social Media Marketing,"Social, Finance, Analytics, Advertising"
5058,Engineering Firms,Others
5641,Consulting,"Social, Finance, Analytics, Advertising"
5467,Translation,Others
3236,Nutrition,Health


In [75]:
#preparing master_frame with Primary Sector
master_frame["primary_sector"] = master_frame.category_list.str.split('|',expand=True)[0]

#mergin the main sector in master_frame and cleaning duplicate binding columns
master_frame = pd.merge(master_frame, main_sector_mapping, how='left', left_on='primary_sector', right_on = 'category_list')
master_frame = master_frame.rename(columns = {'category_list_x':'category_list'})
master_frame.drop(["category_list_y"] , axis = 1, inplace = True)
master_frame.sample(5)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,primary_sector,main_sector
111812,/organization/www-sarmeks-com,/funding-round/11c27c46e0f886e5b142cfa43d102ffe,private_equity,NaN,2013-04-04,0.005,/organization/www-sarmeks-com,Sarmeks Tech,http://www.sarmeks.com,Enterprise Software|Portals|Social Games,operating,UKR,24,UKR - Other,Lutsk,NaT,Enterprise Software,Others
50237,/organization/isirona,/funding-round/1467c56c4d3b6005383e5ddd2a5f083e,venture,NaN,2009-10-28,1.500,/organization/isirona,iSirona,http://isirona.com,Biotechnology,operating,USA,FL,Panama City,Panama City,2007-01-01,Biotechnology,Cleantech / Semiconductors
81973,/organization/recon-instruments,/funding-round/776c1e89e375ca1302c8061190c67bbc,venture,A,2012-01-18,10.000,/organization/recon-instruments,Recon Instruments,http://www.reconinstruments.com,Consumer Electronics|Electronics|Health and We...,acquired,CAN,BC,Vancouver,Vancouver,2008-01-17,Consumer Electronics,Manufacturing
1993,/organization/activegrid,/funding-round/f068ce9f5e13231f780b3c9aae318cfc,venture,A,2004-11-17,3.000,/organization/activegrid,ActiveGrid,http://www.activegrid.com/,Information Technology|SaaS|Visualization,closed,USA,CA,SF Bay Area,San Francisco,NaT,Information Technology,"Social, Finance, Analytics, Advertising"
104513,/organization/ummc,/funding-round/9b8f0afeef2991b4806aff3eb45db49c,debt_financing,NaN,2013-08-22,37.000,/organization/ummc,UMMC,http://umc.edu,Education,operating,USA,MS,Jackson,Jackson,1955-01-01,Education,Others


In [76]:
missing_category_main_sector = master_frame[master_frame.main_sector.isnull()].groupby("primary_sector").primary_sector.count().sort_values(ascending = False)

In [77]:
#lets check if we can solve for main_sector for most commony missing categries, 
(missing_category_main_sector/missing_category_main_sector.sum())[0:5].sum()
#first 5 category covers upto 82% of missing case. also bottom down there are some familiar sector
# that can be tied tomain sector, for ecample Business Analytics to Analytics and Finance Tech to Finance.
print((missing_category_main_sector/missing_category_main_sector.sum())[0:5])
print(main_sector_mapping.main_sector.unique())
master_frame.loc[master_frame.primary_sector.isin(["Analytics", "Business Analytics", "Big Data Analytics", "Finance", "Financial Services", "Finance Technology", "Investment Management"]), "main_sector"]  = "Social, Finance, Analytics, Advertising"
#master_frame.loc[master_frame.primary_sector.isin(["Finance", "Financial Services", "Finance Technology", "Investment Management"]), "main_sector"]  = "Finance"
master_frame.loc[master_frame.primary_sector == "Nanotechnology", "main_sector"]  = "Cleantech / Semiconductors"


primary_sector
Analytics             0.442766
Finance               0.235937
Big Data Analytics    0.056252
Financial Services    0.055760
Nanotechnology        0.031688
Name: primary_sector, dtype: float64
['Automotive & Sports' 'Blanks' 'Cleantech / Semiconductors'
 'Entertainment' 'Health' 'Manufacturing' 'News, Search and Messaging'
 'Others' 'Social, Finance, Analytics, Advertising']


In [78]:
#checking the impact of leftovers on overall investment raised in
round(master_frame[master_frame.main_sector.isnull()].raised_amount_usd.sum()/master_frame.raised_amount_usd.sum(), 3)

0.008

The investements in remainng primary sectors sums upto less than.8%. This is insignificant hence can be grouped too Others

In [79]:
#assigning rest of the missing main sectors to Others
master_frame.loc[master_frame.main_sector.isnull(), "main_sector"] = "Others"

In [80]:
master_frame.info()
master_frame.to_csv("master_frame.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114949 entries, 0 to 114948
Data columns (total 18 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null category
funding_round_code         31140 non-null category
funded_at                  114949 non-null datetime64[ns]
raised_amount_usd          94959 non-null float64
permalink                  114949 non-null object
name                       114948 non-null object
homepage_url               108815 non-null object
category_list              111539 non-null object
status                     114949 non-null category
country_code               107015 non-null category
state_code                 104003 non-null object
region                     104782 non-null object
city                       104785 non-null object
founded_at                 94286 non-null datetime64[ns]
primary_sector             111539 non-null object
main_sector      

## Sector Analysis 2

In [81]:
#check once again the top invested coutries
master_df_venture = master_frame[master_frame.funding_round_type == "venture"]
round(master_df_venture.groupby("country_code").raised_amount_usd.sum().sort_values(ascending = False).head(10), 2)

country_code
USA    422516.84
CHN     40164.30
GBR     20309.70
IND     14394.56
CAN      9585.53
FRA      7298.70
ISR      6937.01
DEU      6346.96
JPN      3398.14
SWE      3255.77
Name: raised_amount_usd, dtype: float64

In [82]:
#1. Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. The three data frames should contain:
#a. All the columns of the master_frame along with the primary sector and the main sector

#function to create country specific constrained dataframes
def get_country_df(df, c_code, fund_type, inv_low, inv_high):
    df = df.loc[(df.country_code == c_code) & (df.funding_round_type == fund_type) & (df.raised_amount_usd.between(inv_low, inv_high, inclusive=True))]
    df
    return df;

In [83]:
#get country specific dataframes
df_usa = get_country_df(master_frame, "USA", "venture", 5, 15)
df_gbr = get_country_df(master_frame, "GBR", "venture", 5, 15)
df_ind = get_country_df(master_frame, "IND", "venture", 5, 15)

In [84]:
#b. The total number (or count) of investments for each main sector in a separate column
#c. The total amount invested in each main sector in a separate column

#function to calculate the metric b and c
def get_m_sec_metric(df):
    df = pd.merge(df, df.groupby("main_sector").sum().rename(columns={"raised_amount_usd":"total_investement_amount"}), how = "left", left_on = "main_sector", right_on = "main_sector")
    df = pd.merge(df, df.groupby("main_sector").raised_amount_usd.count().to_frame().rename(columns={"raised_amount_usd":"total_investement_num"}), how = "left", left_on = "main_sector", right_on = "main_sector")
    df
    return df;

In [85]:
#get sector metric
df_usa = get_m_sec_metric(df_usa)
df_gbr = get_m_sec_metric(df_gbr)
df_ind = get_m_sec_metric(df_ind)

df_usa.to_csv("df_usa.csv", index=False)
df_gbr.to_csv("df_gbr.csv", index=False)
df_ind.to_csv("df_ind.csv", index=False)






### Country_Sector-wise Investment Analysis-Metrics for Table 5.1

In [86]:
def get_country_sector_metrics(df):
    print("Total number of investments")
    print(df.funding_round_type.count())
    print("Total amount of investment (USD)")
    #the above two metric not necessarily has to have a symmetric mapping.
    #As both  may independently answer the two indpendent business question. 
    print(round(df.raised_amount_usd.sum()), 2)
    print("Top Sector name (no. of investment-wise)")
    top_3 = df.groupby("main_sector").main_sector.count().to_frame().sort_values(by = "main_sector", ascending = False).rename(columns={"main_sector":"no. of investment"}).head(3)
    print(top_3)
    print("\nCompany receiving the highest investment in " + top_3.index[0] + ":")

    print(round(df[df.main_sector == 
            top_3.index[0]].groupby(["company_permalink", "name"]).raised_amount_usd.sum().sort_values(ascending = False), 2).to_frame().index[0][1])
    
    print("\nCompany receiving the highest investment in " + top_3.index[1] + ":")
    print(round(df[df.main_sector == 
           top_3.index[1]].groupby(["company_permalink", "name"]).raised_amount_usd.sum().sort_values(ascending = False), 2).to_frame().index[0][1])


In [87]:
#get metrics for USA
get_country_sector_metrics(df_usa)

Total number of investments
12151
Total amount of investment (USD)
108536.0 2
Top Sector name (no. of investment-wise)
                                         no. of investment
main_sector                                               
Others                                                3028
Social, Finance, Analytics, Advertising               2672
Cleantech / Semiconductors                            2347

Company receiving the highest investment in Others:
Virtustream

Company receiving the highest investment in Social, Finance, Analytics, Advertising:
SST Inc. (Formerly ShotSpotter)


In [88]:
#get metrics for Great Britain
get_country_sector_metrics(df_gbr)

Total number of investments
630
Total amount of investment (USD)
5455.0 2
Top Sector name (no. of investment-wise)
                                         no. of investment
main_sector                                               
Others                                                 153
Social, Finance, Analytics, Advertising                130
Cleantech / Semiconductors                             129

Company receiving the highest investment in Others:
Electric Cloud

Company receiving the highest investment in Social, Finance, Analytics, Advertising:
Celltick Technologies


In [89]:
#Get metrics for India
get_country_sector_metrics(df_ind)

Total number of investments
330
Total amount of investment (USD)
2977.0 2
Top Sector name (no. of investment-wise)
                                         no. of investment
main_sector                                               
Others                                                 110
Social, Finance, Analytics, Advertising                 60
News, Search and Messaging                              52

Company receiving the highest investment in Others:
FirstCry.com

Company receiving the highest investment in Social, Finance, Analytics, Advertising:
Manthan Systems
